# PyMem3DG Tutorial 7 - Unwrapped integrator

It is possible to write an integrator directly in Python such that there is more control over its behavior. It could also be useful if you want to modify certain parameters/data during simulation. Here is a snippet on how to write a simple forward Euler integrator. 

In [1]:
import pymem3dg as dg
oblate = "sample_meshes/oblate.ply"
p = dg.Parameters()
p.bending.Kbc = 8.22e-5
p.tension.Ksg = 0.1
p.tension.At = 12.4866
p.osmotic.isPreferredVolume = True
p.osmotic.Kv = 0.02
p.osmotic.Vt = 0.7 * 3.14 * 4 / 3

In [2]:
g = dg.System(inputMesh=oblate, parameters=p)
g.initialize()

outputDir = "output/tutorial7"
fe = dg.Euler(system=g, characteristicTimeStep=1,
              tolerance=1e-5, outputDirectory=outputDir)

frame = 0
lastSave = g.time
lastProcessMesh = g.time
initTime = g.time
tSave = 2000 * fe.characteristicTimeStep
processMeshPeriod = 30

fe.integrate()
fe.createMutableNetcdfFile(isContinue=False)
while frame < 10:
    # compute forces/energy, and test for fe.EXIT
    fe.status()

    # saveData
    if ((g.time - lastSave > tSave) | (g.time == initTime) | fe.EXIT):
        lastSave = g.time
        fe.saveData(ifOutputTrajFile=True,
                    ifOutputMeshFile=False, ifPrintToConsole=True)
        frame = frame + 1

    if (fe.EXIT):
        break

    # mesh processing
    if (g.time - lastProcessMesh > (processMeshPeriod * fe.timeStep)):
        lastProcessMesh = g.time
        g.mutateMesh(1)
        g.updateConfigurations()

    # time stepping and update variables
    if (g.time == lastProcessMesh):
        g.time = g.time + 1e-10 * fe.characteristicTimeStep
    else:
        fe.march()

fe.closeMutableNetcdfFile()


In [3]:
import pymem3dg.visual as dg_vis
dg_vis.animate(outputDir+"/traj.nc", meanCurvature = True)